# 第三章，函数

### 第19条，不要把函数返回的多个数值拆分到三个以上的变量中

### 第20条，遇到意外情况要返回异常，不要返回None

### 第21条，了解如何在闭包里面使用外围作用域中的变量

In [6]:
# closure 闭包
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return(1, x)
    values.sort(key=helper)

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


In [7]:
# 当需要将闭包里的数据赋给闭包外面的变量时，要使用nonlocal
# nonlocal使用有限制，不能侵入模块级别的作用域
def sort_priority2(values, group):
    found = False
    def helper(x):
        nonlocal found  # 添加该变量
        if x in group:
            found = True
            return(0, x)
        return(1, x)
    values.sort(key=helper)
    return found
    
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority2(numbers, group)
print(found)

# nonlocal尽量少用

True


### 第22条，用数量可变的位置参数给函数设计清晰的参数列表

In [12]:
# 数量可变的位置参数，简称varargs，叫做star args或*args
# 以下为记录调试信息的函数

# 不使用可变参数版本
def log(message, values):
    if not values:
        print(message)
    else:
        value_str = ', '.join(str(x) for x in values)
        print(f'{message}: {value_str}')

log('My numbers are', [1,2])
log('Hi there', [])             # 这里必须加一个空列表作为values的参数

My numbers are: 1, 2
Hi there


In [15]:
# 使用可变参数版本
def log(message, *values):
    if not values:
        print(message)
    else:
        value_str = ', '.join(str(x) for x in values)
        print(f'{message}: {value_str}')

log('My numbers are', 1, 2)
log('Hi there')                     # 这里不用加参数
numbers = [1,2]
log('My numbers are', numbers)      # 注意传参时，加*与不加*的区别
log('My numbers are', *numbers)

My numbers are: 1, 2
Hi there
My numbers are: [1, 2]
My numbers are: 1, 2


In [19]:
d = {'a':1, 'b':2}
# **d，作为关键字参数传给函数，等价于'a'=1, 'b'=2

### 第24条，用None和docstrinng来描述默认值会变的参数

In [1]:
# 函数的参数是在系统加载该函数模块时计算一遍，不会在每次执行时重新计算
from datetime import datetime
def log(message, when = None):
    '''
    Log a message with a timestamp

    Args:
        message: message to print。
        when： datetime of when the message occurred
    '''
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

### 第25条，用只能以关键字，或只能以按位置传入参数的方法，为函数设计清晰的参数列表

In [6]:
# 可以用*分割参数，*左边是位置参数，右边是只能用关键字制定参数
# 3.8版本后新增功能，可以用/分割参数，/左边必须使用按位置制定参数
# /和*之间的参数，既可以按位置提供，也可以按关键字提供

def print_func(arg1, arg2, /, *, kwag1, kwag2):
    print(arg1, arg2, kwag1, kwag2)

# print_func(0, 1, 2, 3)                        # 会报错
# print_func(arg1=0, arg2=1, kwag1=2, kwag2=3)  # 会报错
print_func(0, 1, kwag1=2, kwag2=3)              # 成功

0 1 2 3


### 第26条，用functools.wraps定义函数修饰器

In [7]:
def trace(func):
    def wrapper(*args, **kwags):
        result = func(*args, **kwags)       # 获得原函数结果
        print(f'{func.__name__}({args!r}, {kwags!r}) -> {result!r}')        # 修饰原函数
        return result
    return wrapper

@trace      # 等价于fibonacci = trace(fibonacci)
def fibonacci(n):
    '''斐波那契数列的递归实现'''
    if n in (0, 1):
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(4)
print(fibonacci) 
help(fibonacci)

fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((4,), {}) -> 3
<function trace.<locals>.wrapper at 0x0000025C5D452680>
Help on function wrapper in module __main__:

wrapper(*args, **kwags)



- 可以看到，经过装饰的函数，已经变为trace内的wrapper函数，而不是原来的fibonacci函数了
- 但是这样有问题，会影响调试器debugger的使用，也无法访问原函数的帮助文档，也无法使用pickle对象序列化器保存函数

In [9]:
# functools中的wraps模块本身也是个修饰器，可以帮助将重要的元数据从原函数复制到外部函数中
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwags):
        result = func(*args, **kwags)
        print(f'{func.__name__}({args!r}, {kwags!r}) -> {result!r}')
        return result
    return wrapper

@trace
def fibonacci(n):
    '''斐波那契数列的递归实现'''
    if n in (0, 1):
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(4)
print(fibonacci) 
help(fibonacci)

fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((4,), {}) -> 3
<function fibonacci at 0x0000025C5D4528C0>
Help on function fibonacci in module __main__:

fibonacci(n)
    斐波那契数列递归实现

